In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)


In [4]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#トレーニングデータ準備
train_file = unpickle("data_batch_1")
labels = train_file[b'labels']

data = train_file[b'data']
data = data.reshape([-1,3,32,32]).transpose([0,2,3,1])
data = data.reshape([-1,3*32*32])

#トレーニング、ラベルをonehot にする
labels_oh = np.zeros([len(labels),10])
labels_oh[np.arange(len(labels)),labels]  =1

#テストデータ準備
test_file = unpickle("test_batch")
test_labels = test_file[b'labels']
test_data = test_file[b'data']

#テスト、ラベルをonehot にする
test_labels_oh = np.zeros([len(test_labels),10])
test_labels_oh[np.arange(len(test_labels)),test_labels]  =1



print(data.shape)
print(labels_oh.shape)
print(test_data.shape)
print(test_labels_oh.shape)
print(data.shape)

(10000, 3072)
(10000, 10)
(10000, 3072)
(10000, 10)
(10000, 3072)


In [ ]:

x = tf.placeholder(tf.float32, [None,3072])
x_images = tf.reshape(x,[-1,32,32,3])

num_filter1 = 32
w1_conv = tf.Variable(tf.truncated_normal([5,5,3,num_filter1], stddev = 0.1))
b1_conv = tf.Variable(tf.constant(0.1, shape=[num_filter1]))
h1_conv = tf.nn.conv2d(x_images, w1_conv, strides=[1,1,1,1], padding="SAME")
h1_conv_cutoff = tf.nn.relu(h1_conv+b1_conv)
h1_pool = tf.nn.max_pool(h1_conv_cutoff, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")

num_filter2 = 64
w2_conv = tf.Variable(tf.truncated_normal([5,5,num_filter1 ,num_filter2], stddev = 0.1))
b2_conv = tf.Variable(tf.constant(0.1, shape=[num_filter2]))
h2_conv = tf.nn.conv2d(h1_pool, w2_conv, strides=[1,1,1,1], padding="SAME")
h2_conv_cutoff = tf.nn.relu(h2_conv+b2_conv)
h2_pool = tf.nn.max_pool(h2_conv_cutoff, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")
h2_pool_flat = tf.reshape(h2_pool, [-1, 8*8*num_filter2])

num_units1 = 8*8*num_filter2
num_units2 = 1024

w2 =  tf.Variable(tf.truncated_normal([num_units1 ,num_units2], stddev = 0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h2_pool_flat,w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden_drop =  tf.nn.dropout(hidden2, keep_prob)
w0 = tf.Variable(tf.truncated_normal([num_units2, 10],stddev = 0.1))
b0 = tf.Variable(tf.constant(0.1, shape=[10]))
p = tf.nn.softmax(tf.matmul(hidden_drop,w0) + b0)

t = tf.placeholder(tf.float32, [None, 10])
correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(t,1))
loss = -tf.reduce_sum(t*tf.log(p))
train = tf.train.AdamOptimizer(0.0001).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

i = 0
for _ in range(10000):
    i += 1
    sess.run(train, feed_dict={x: data[i:i+1,:], t: labels_oh[i:i+1,:], keep_prob:0.5})   
    if i%100 == 0:
        loss_val, acc_val = [],[]
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:test_data, t:test_labels_oh, keep_prob:1.0})        
        print ('Step: %d, Loss: %f, Accuracy: %f'% (i, loss_val, acc_val))


Step: 100, Loss: nan, Accuracy: 0.100000
Step: 200, Loss: nan, Accuracy: 0.100000
Step: 300, Loss: nan, Accuracy: 0.100000
Step: 400, Loss: nan, Accuracy: 0.100000
Step: 500, Loss: nan, Accuracy: 0.100000
Step: 600, Loss: nan, Accuracy: 0.100000
Step: 700, Loss: nan, Accuracy: 0.100000
Step: 800, Loss: nan, Accuracy: 0.100000
Step: 900, Loss: nan, Accuracy: 0.100000
Step: 1000, Loss: nan, Accuracy: 0.100000
Step: 1100, Loss: nan, Accuracy: 0.100000
Step: 1200, Loss: nan, Accuracy: 0.100000


In [6]:
data[1:1+1,:].shape

(1, 3072)